# Subsetting on Arbitrary Dimensions

### Environment Setup
Install the required packages in your virtual environment and import them

In [ ]:
import sys
!{sys.executable} -m pip install netCDF4 harmony-py

In [2]:
from harmony import BBox, Client, Collection, Request, Dimension
from harmony.config import Environment
import os
import netCDF4 as nc4

### Request Helper Function

Submits the Harmony request via harmony-py, downloads to a local file, and returns the result file name

In [3]:
def submit_request(request, harmony_client, output_dir):
    job_id = harmony_client.submit(request)
    harmony_client.wait_for_processing(job_id, show_progress=True)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    results = harmony_client.download_all(job_id, directory=output_dir, overwrite=True)
    file_names = [f.result() for f in results]
    return file_names[0]

### Request Execution with harmony-py

Construct the Harmony request, subsetting on **"lev"** and a bounding box

In [4]:
collection = Collection(id='C1215726323-GES_DISC')
request = Request(
    collection = collection,
    spatial    = BBox(-160, -10, -75, 10),
    dimensions = [Dimension(name='lev', min=800, max=900)],
    max_results = 1
)

Submit the request to Harmony via harmony-py and output the result to a local directory

In [5]:
harmony_client = Client(env=Environment.UAT) # assumes .netrc usage
harmony_fname = submit_request(request, harmony_client, './harmony_dimension_subset_results')

 [ Processing:   0% ] |                                                   | [/]
 [ Processing:   0% ] |                                                   | [-]
 [ Processing:   0% ] |                                                   | [\]
 [ Processing:   0% ] |                                                   | [|]
 [ Processing:   0% ] |                                                   | [/]
 [ Processing:   0% ] |                                                   | [-]
 [ Processing:   0% ] |                                                   | [\]
 [ Processing:   0% ] |                                                   | [|]
 [ Processing:   0% ] |                                                   | [/]
 [ Processing:   0% ] |                                                   | [-]
 [ Processing:   0% ] |                                                   | [\]
 [ Processing:   0% ] |                                                   | [|]
 [ Processing:   0% ] |                 

### Request Results
Inspect the results to see how the data has been subsetted

In [6]:
nc4_file=nc4.Dataset(harmony_fname)
print(nc4_file.variables['lev'][:])
print(min(nc4_file.variables['lev']))
print(max(nc4_file.variables['lev']))

[900. 875. 850. 825. 800.]
800.0
900.0
